# 2D Taylor Green Vortex

## Overview

In fluid dynamics, the Taylor–Green vortex is an unsteady flow of a decaying vortex, which has an exact closed form solution of the incompressible Navier–Stokes equations in Cartesian coordinates. It is named after the British physicist and mathematician Geoffrey Ingram Taylor and his collaborator A. E. Green.

Physics-informed Neural Networks (PINNs) provides a new method for quickly solving complex fluid problems by using loss functions that approximate governing equations coupled with simple network configurations. In this case, the data-driven characteristic of neural network is used along with `PINNs` to solve the 2D taylor green vortex problem

## Problem Description

The Navier-Stokes equation, referred to as `N-S` equation, is a classical partial differential equation in the field of fluid mechanics. In the case of viscous incompressibility, the dimensionless `N-S` equation has the following form:

$$
\frac{\partial u}{\partial x} + \frac{\partial v}{\partial y} = 0
$$

$$
\frac{\partial u} {\partial t} + u \frac{\partial u}{\partial x} + v \frac{\partial u}{\partial y} = - \frac{\partial p}{\partial x} + \frac{1} {Re} (\frac{\partial^2u}{\partial x^2} + \frac{\partial^2u}{\partial y^2})
$$

$$
\frac{\partial v} {\partial t} + u \frac{\partial v}{\partial x} + v \frac{\partial v}{\partial y} = - \frac{\partial p}{\partial y} + \frac{1} {Re} (\frac{\partial^2v}{\partial x^2} + \frac{\partial^2v}{\partial y^2})
$$

where `Re` stands for Reynolds number.

In this case, the PINNs method is used to learn the mapping from the location and time to flow field quantities to solve the `N-S` equation.

$$
(x, y, t) \mapsto (u, v, p)
$$

## Technology Path

MindFlow solves the problem as follows:

1. Training Dataset Construction.
2. Model Construction.
3. Multi-task Learning for Adaptive Losses
4. Optimizer.
5. NavierStokes2D.
6. Model Training.
7. Model Evaluation and Visualization.

## Import necessary package

In [1]:
import time
import numpy as np
import sympy
import mindspore
from mindspore import context, nn, ops, jit, set_seed
from mindspore import numpy as mnp

from mindflow.cell import MultiScaleFCCell
from mindflow.loss import MTLWeightedLossCell
from mindflow.utils import load_yaml_config
from mindflow.pde import NavierStokes, sympy_to_mindspore

from src import create_training_dataset, create_test_dataset, calculate_l2_error, NavierStokes2D

set_seed(123456)
np.random.seed(123456)

context.set_context(mode=context.GRAPH_MODE, device_target="GPU", device_id=0, save_graphs=False)
use_ascend = context.get_context(attr_key='device_target') == "Ascend"

config = load_yaml_config('taylor_green_2D.yaml')

## Dataset Construction

Training dataset is imported through function _create_train_dataset_, contains domain points, initial condition points and boundary condition point. All datasets are sampled by APIs from mindflow.geometry.

Test dataset is imported through function _create_test_dataset_. In this case, the exact solution used to construct test dataset is given by **J Kim, P Moin,Application of a fractional-step method to incompressible Navier-Stokes equations,Journal of Computational Physics,Volume 59, Issue 2,1985**.

$$
u(x,y,t) = -cos(x)sin(y)e^{-2t}
$$

$$
v(x,y,t) = sin(x)cos(y)e^{-2t}
$$

$$
p(x,y,t) = -0.25(cos(2x)+cos(2y))e^{-4t}
$$

The computation is carried out in the domain of $ 0 \leq  x,y \leq 2\pi$,
and $ 0 \leq t \leq 2$. The Reynolds number Re is equal to 1

In [2]:
# create training dataset
taylor_dataset = create_training_dataset(config)
train_dataset = taylor_dataset.create_dataset(batch_size=config["train_batch_size"],
                                              shuffle=True,
                                              prebatched_data=True,
                                              drop_remainder=True)

# create test dataset
inputs, label = create_test_dataset(config)

## Model Construction

This example uses a simple fully-connected network with a depth of 6 layers and the activation function is the `tanh` function.

In [3]:
coord_min = np.array(config["geometry"]["coord_min"] + [config["geometry"]["time_min"]]).astype(np.float32)
coord_max = np.array(config["geometry"]["coord_max"] + [config["geometry"]["time_max"]]).astype(np.float32)
input_center = list(0.5 * (coord_max + coord_min))
input_scale = list(2.0 / (coord_max - coord_min))

model = MultiScaleFCCell(in_channels=config["model"]["in_channels"],
                         out_channels=config["model"]["out_channels"],
                         layers=config["model"]["layers"],
                         neurons=config["model"]["neurons"],
                         residual=config["model"]["residual"],
                         act='tanh',
                         num_scales=1,
                         input_scale=input_scale,
                         input_center=input_center)

## Multi-task learning for adaptive losses

The PINNs method needs to optimize multiple losses at the same time, and brings challenges to the optimization process. Here, we adopt the uncertainty weighting algorithm proposed in **Kendall, Alex, Yarin Gal, and Roberto Cipolla. "Multi-task learning using uncertainty to weigh losses for scene geometry and semantics." CVPR, 2018.** to dynamically adjust the weights.

In [4]:
mtl = MTLWeightedLossCell(num_losses=taylor_dataset.num_dataset)
print("Use MtlWeightedLossCell, num loss: {}".format(mtl.num_losses))

Use MtlWeightedLossCell, num loss: 3


## Optimizer

In [5]:
params = model.trainable_params() + mtl.trainable_params()
optimizer = nn.Adam(params, learning_rate=config["optimizer"]["initial_lr"])

## NavierStokes2D

The following `NavierStokes2D` defines the navier-stokes' problem. Specifically, it includes 3 parts: governing equation, initial condition and boundary conditions. Initial condition and boundary conditions are constructed by the exact solution mentioned before.

In [6]:
class NavierStokes2D(NavierStokes):
    def __init__(self, model, re=100, loss_fn=nn.MSELoss()):
        super(NavierStokes2D, self).__init__(model, re=re, loss_fn=loss_fn)
        self.ic_nodes = sympy_to_mindspore(self.ic(), self.in_vars, self.out_vars)
        self.bc_nodes = sympy_to_mindspore(self.bc(), self.in_vars, self.out_vars)

    def ic(self):
        """
        Define initial condition equations based on sympy, abstract method.
        """
        ic_u = self.u + sympy.cos(self.x) * sympy.sin(self.y)
        ic_v = self.v - sympy.sin(self.x) * sympy.cos(self.y)
        ic_p = self.p + 0.25 * (sympy.cos(2*self.x) + sympy.cos(2*self.y))
        equations = {"ic_u": ic_u, "ic_v": ic_v, "ic_p": ic_p}
        return equations

    def bc(self):
        """
        Define boundary condition equations based on sympy, abstract method.
        """
        bc_u = self.u + sympy.cos(self.x) * sympy.sin(self.y) * sympy.exp(-2*self.t)
        bc_v = self.v - sympy.sin(self.x) * sympy.cos(self.y) * sympy.exp(-2*self.t)
        bc_p = self.p + 0.25 * (sympy.cos(2*self.x) + sympy.cos(2*self.y)) * sympy.exp(-4*self.t)
        equations = {"bc_u": bc_u, "bc_v": bc_v, "bc_p": bc_p}
        return equations

    def get_loss(self, pde_data, ic_data, bc_data):
        """
        Compute loss of 3 parts: governing equation, initial condition and boundary conditions.

        Args:
            pde_data (Tensor): the input data of governing equations.
            ic_data (Tensor): the input data of initial condition.
            bc_data (Tensor): the input data of boundary condition.
        """
        pde_res = self.parse_node(self.pde_nodes, inputs=pde_data)
        pde_residual = ops.Concat(1)(pde_res)
        pde_loss = self.loss_fn(pde_residual, mnp.zeros_like(pde_residual))

        ic_res = self.parse_node(self.ic_nodes, inputs=ic_data)
        ic_residual = ops.Concat(1)(ic_res)
        ic_loss = self.loss_fn(ic_residual, mnp.zeros_like(ic_residual))

        bc_res = self.parse_node(self.bc_nodes, inputs=bc_data)
        bc_residual = ops.Concat(1)(bc_res)
        bc_loss = self.loss_fn(bc_residual, mnp.zeros_like(bc_residual))

        return pde_loss + ic_loss + bc_loss

## Model Training

With **MindSpore version >= 2.0.0**, we can use the functional programming for training neural networks.

In [7]:
def train():
    problem = NavierStokes2D(model, re=config["Re"])

    if use_ascend:
        from mindspore.amp import DynamicLossScaler, auto_mixed_precision, all_finite
        loss_scaler = DynamicLossScaler(1024, 2, 100)
        auto_mixed_precision(model, 'O3')

    def forward_fn(pde_data, ic_data, bc_data):
        loss = problem.get_loss(pde_data, ic_data, bc_data)
        if use_ascend:
            loss = loss_scaler.scale(loss)
        return loss

    grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=False)

    @jit
    def train_step(pde_data, ic_data, bc_data):
        loss, grads = grad_fn(pde_data, ic_data, bc_data)
        if use_ascend:
            loss = loss_scaler.unscale(loss)
            if all_finite(grads):
                grads = loss_scaler.unscale(grads)
                loss = ops.depend(loss, optimizer(grads))
        else:
            loss = ops.depend(loss, optimizer(grads))
        return loss

    steps = config["train_steps"]

    sink_process = mindspore.data_sink(train_step, train_dataset, sink_size=1)
    model.set_train()
    for step in range(steps + 1):
        local_time_beg = time.time()
        cur_loss = sink_process()
        if step % 1000 == 0:
            print(f"loss: {cur_loss.asnumpy():>7f}")
            print("step: {}, time elapsed: {}ms".format(step, (time.time() - local_time_beg) * 1000))
            calculate_l2_error(model, inputs, label, config)

In [ ]:
time_beg = time.time()
train()
print("End-to-End total time: {} s".format(time.time() - time_beg))

```python
check training dataset size:  128
Use MtlWeightedLossCell, num loss: 3
momentum_x: u(x, y, t)*Derivative(u(x, y, t), x) + v(x, y, t)*Derivative(u(x, y, t), y) + Derivative(p(x, y, t), x) + Derivative(u(x, y, t), t) - 1.0*Derivative(u(x, y, t), (x, 2)) - 1.0*Derivative(u(x, y, t), (y, 2))
    Item numbers of current derivative formula nodes: 6
momentum_y: u(x, y, t)*Derivative(v(x, y, t), x) + v(x, y, t)*Derivative(v(x, y, t), y) + Derivative(p(x, y, t), y) + Derivative(v(x, y, t), t) - 1.0*Derivative(v(x, y, t), (x, 2)) - 1.0*Derivative(v(x, y, t), (y, 2))
    Item numbers of current derivative formula nodes: 6
continuty: Derivative(u(x, y, t), x) + Derivative(v(x, y, t), y)
    Item numbers of current derivative formula nodes: 2
ic_u: u(x, y, t) + sin(y)*cos(x)
    Item numbers of current derivative formula nodes: 2
ic_v: v(x, y, t) - sin(x)*cos(y)
    Item numbers of current derivative formula nodes: 2
ic_p: p(x, y, t) + 0.25*cos(2*x) + 0.25*cos(2*y)
    Item numbers of current derivative formula nodes: 3
bc_u: u(x, y, t) + exp(-2*t)*sin(y)*cos(x)
    Item numbers of current derivative formula nodes: 2
bc_v: v(x, y, t) - exp(-2*t)*sin(x)*cos(y)
    Item numbers of current derivative formula nodes: 2
bc_p: p(x, y, t) + 0.25*exp(-4*t)*cos(2*x) + 0.25*exp(-4*t)*cos(2*y)
    Item numbers of current derivative formula nodes: 3
loss: 0.218985
step: 0, time elapsed: 8551.078081130981ms
    predict total time: 395.74146270751953 ms
    l2_error, U:  1.0272722185259853 , V:  1.0068734156826729 , P:  1.1140811724988493 , Total:  1.025431141488198
==================================================================================================
loss: 0.117311
step: 1000, time elapsed: 96.29058837890625ms
    predict total time: 177.2327423095703 ms
    l2_error, U:  0.7230177964964256 , V:  0.7154075373127421 , P:  1.0178466945703284 , Total:  0.7482490912542953
==================================================================================================
loss: 0.119411
step: 2000, time elapsed: 89.85757827758789ms
    predict total time: 143.2340145111084 ms
    l2_error, U:  0.7112490531604364 , V:  0.7168233752672813 , P:  1.0153781956196177 , Total:  0.7434033117415322
==================================================================================================
loss: 0.116417
step: 3000, time elapsed: 88.8054370880127ms
    predict total time: 137.58182525634766 ms
    l2_error, U:  0.7190401305154112 , V:  0.7173188474021113 , P:  0.9807884344618322 , Total:  0.7432489940800948
==================================================================================================
......
.....
loss: 0.000172
step: 29000, time elapsed: 87.26811408996582ms
    predict total time: 121.50454521179199 ms
    l2_error, U:  0.030043695657698256 , V:  0.02123015788094162 , P:  0.05539040596455253 , Total:  0.029547291823711123
==================================================================================================
loss: 0.000119
step: 30000, time elapsed: 89.66708183288574ms
    predict total time: 171.91457748413086 ms
    l2_error, U:  0.015388734902569713 , V:  0.012925750524951195 , P:  0.04789422732887809 , Total:  0.019331853743076763
==================================================================================================
```

## Model Evaluation and Visualization

In [ ]:
from src import visualization

# visualization
visualization(model=model, step=config["train_steps"], input_data=inputs, label=label)

![time-error](./images/TimeError_30000.png "error after training")

As the speed tends to decrease exponentially, the error becomes larger with time, but the overall is within the 5% error range. Picture below shows u, v, p during the process.

![mid_stage](./images/mid_stage.png "mid stage status")